In [1]:
#!pip install mysql.connector

In [2]:
#!pip install Flask
#!pip install flask_ngrok

In [3]:
#!pip install flask flask-sqlalchemy flask-login

In [4]:
#!pip install bs4


In [5]:
#import lib
from flask_ngrok import run_with_ngrok
from flask import Flask
from flask import Flask, render_template,redirect,url_for
from flask import Blueprint
from flask import request
from flask import flash
from flask import session
import secrets
import flask
from datetime import date
from datetime import datetime
import json
import time 
from flask import send_file
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv

In [6]:
import mysql.connector 
from mysql.connector import errorcode
import mysql.connector
#connection db config
config = {
  'user': 'samson',
  'password': 'samson0214',
  'host': '35.188.147.97',
  'database': 'project_db',
  'raise_on_warnings': True
}
try:
  mydb = mysql.connector.connect(**config)
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)

In [9]:
app = Flask(__name__)
app.secret_key = secrets.token_urlsafe(16)
#run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/")
def home():
  return render_template('UserLogin.html')

@app.route("/userlogin",methods=["POST"])
def login_post():
    email = request.form.get('Email')
    password = request.form.get('Password')
    sql =  "SELECT * FROM User_AC WHERE Email = %s AND Password = %s"
    # if this returns a user, then the email already exists in database
    mycursor = mydb.cursor()
    mycursor.execute(sql,(email,password,))
    myresult = mycursor.fetchall()
    if not myresult: # if a user is found, we want to redirect back to signup page so user can try again
      flash('This is account not exits')
      return render_template('UserLogin.html')
    mydb.commit()    
    session['userID'] = myresult[0][0]#user id 
    return  redirect(url_for('index'))

@app.route("/userSignup",methods=["POST","GET"])
def userSignup():
    if request.method =='GET':
        return render_template('userSignup.html')
    else:
        email = request.form.get('Email')
        name = request.form.get('name')
        password = request.form.get('Password')    
        sql = "SELECT * FROM User_AC WHERE Email = %s"
        mycursor = mydb.cursor()      
        mycursor.execute(sql,(email,))
        myresult = mycursor.fetchall()
        if myresult: # if a user is found, we want to redirect back to signup page so user can try again      
          flash('This is account alredy exits')
          return render_template('userSignup.html')
        else:
          sql = "INSERT INTO User_AC(name,email, password) VALUES (%s, %s, %s)"
          mycursor.execute(sql,(name,email,password,))
          mydb.commit()
          #get new user id 
          sql =  "SELECT * FROM User_AC WHERE Email = %s AND Password = %s"
          mycursor = mydb.cursor()
          mycursor.execute(sql,(email,password,))
          myresult = mycursor.fetchall()
          session['userID'] = myresult[0][0]
          return redirect(url_for('index'))

@app.route("/userChangePW",methods=["POST","GET"])
def userChangePW():
    if request.method =='GET':
        return render_template('account_tools.html')
    else:
        #email = request.form.get('email')    
        password = request.form.get('password')
        userid=flask.escape(session['userID'])
        new_password= request.form.get('new_Password')
        sql = "SELECT * FROM User_AC WHERE id = %s"
        mycursor = mydb.cursor()      
        mycursor.execute(sql,(str(userid),))
        myresult = mycursor.fetchall()
        mydb.commit()
        user_old_password=myresult[0][2]#old password   
        if user_old_password != password:
            flash('The password is not match!')
            return render_template('account_tools.html')  
        else:
            sql="UPDATE User_AC SET Password= %s where id= %s"
            mycursor=mydb.cursor()
            mycursor.execute(sql,(str(new_password),str(userid),))
            mydb.commit()
            flash('The password is changed!')
            return redirect(url_for('logout'))
      
#not use   
@app.route("/delect_user")
def delect_acc():
         userid=flask.escape(session['userID'])
         sql = " DELETE FROM User_AC  WHERE  id = %s"
         mycursor=mydb.cursor()
         mycursor.execute(sql,(str(userid),))
         mydb.commit()
         session["userID"] = None
         return redirect(url_for('/'))

@app.route("/user_logout",methods=["POST","GET"])
def logout():
         session["userID"] = None
         return render_template('UserLogin.html')    

@app.route("/index")
def index():
        arr=[];
        c_day=str(date.today())
        sql="SELECT img_Link,product_img.Pname,W_price,P_price,product_img.p_id FROM Product_record,product_img where product_img.Pname = Product_record.Pname and Product_record.lastDate= %s and type =1;"
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(c_day),))
        myresult=mycursor.fetchall()
        arr.append(myresult)
        mydb.commit() 
        sql="SELECT img_Link,product_img.Pname,W_price,P_price,product_img.p_id FROM Product_record,product_img where product_img.Pname = Product_record.Pname and Product_record.lastDate= %s and type =2;"
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(c_day),))
        myresult=mycursor.fetchall()
        mydb.commit() 
        arr.append(myresult)
        sql="SELECT img_Link,product_img.Pname,W_price,P_price,product_img.p_id FROM Product_record,product_img where product_img.Pname = Product_record.Pname and Product_record.lastDate= %s and type =3;"
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(c_day),))
        myresult=mycursor.fetchall()
        mydb.commit() 
        arr.append(myresult)
        sql="SELECT img_Link,product_img.Pname,W_price,P_price,product_img.p_id FROM Product_record,product_img where product_img.Pname = Product_record.Pname and Product_record.lastDate= %s and type =4;"
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(c_day),))
        myresult=mycursor.fetchall()
        mydb.commit() 
        arr.append(myresult)
        sql="SELECT img_Link,product_img.Pname,W_price,P_price,product_img.p_id FROM Product_record,product_img where product_img.Pname = Product_record.Pname and Product_record.lastDate= %s and type =5;"
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(c_day),))
        myresult=mycursor.fetchall()
        mydb.commit() 
        arr.append(myresult)
        return render_template('show_product.html',data=arr)
    

@app.route("/index_beverage")
def index_beverage():
        c_day=str(date.today())
        sql="SELECT img_Link,product_img.Pname,W_price,P_price,product_img.p_id,type FROM Product_record,product_img where product_img.Pname = Product_record.Pname and Product_record.lastDate= %s and type =1;"
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(c_day),))
        myresult=mycursor.fetchall()
        mydb.commit() 
        return render_template('show_product_cat.html',data=myresult)
@app.route("/index_Egg")
def index_Egg():
        c_day=str(date.today())
        sql="SELECT img_Link,product_img.Pname,W_price,P_price,product_img.p_id,type FROM Product_record,product_img where product_img.Pname = Product_record.Pname and Product_record.lastDate= %s and type =2"
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(c_day),))
        myresult=mycursor.fetchall()
        mydb.commit() 
        return render_template('show_product_cat.html',data=myresult)

@app.route("/index_Rice")
def index_Rice():
        c_day=str(date.today())
        sql="SELECT img_Link,product_img.Pname,W_price,P_price,product_img.p_id,type FROM Product_record,product_img where product_img.Pname = Product_record.Pname and Product_record.lastDate= %s and type =3"
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(c_day),))
        myresult=mycursor.fetchall()
        mydb.commit() 
        return render_template('show_product_cat.html',data=myresult)

@app.route("/index_Tissue")
def index_Tissue():
        c_day=str(date.today())
        sql="SELECT img_Link,product_img.Pname,W_price,P_price,product_img.p_id,type FROM Product_record,product_img where product_img.Pname = Product_record.Pname and Product_record.lastDate= %s and type =4"
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(c_day),))
        myresult=mycursor.fetchall()
        mydb.commit() 
        return render_template('show_product_cat.html',data=myresult)

@app.route("/index_Oil")
def index_Oil():
        c_day=str(date.today())
        sql="SELECT img_Link,product_img.Pname,W_price,P_price,product_img.p_id,type FROM Product_record,product_img where product_img.Pname = Product_record.Pname and Product_record.lastDate= %s and type =5"
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(c_day),))
        myresult=mycursor.fetchall()
        mydb.commit() 
        return render_template('show_product_cat.html',data=myresult)

@app.route("/product_detail")
def product_detail():
        arr=[]
        c_day=str(date.today())
        p_id=request.args.get('id')    
        sql="SELECT img_Link,product_img.Pname,W_price,P_price,product_img.p_id,type,W_price_remark,W_price_avg_remark,P_price_remark,P_price_avg_remark FROM Product_record,product_img where product_img.Pname = Product_record.Pname and Product_record.lastDate= %s and product_img.p_id=%s  "
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(c_day),str(p_id),))
        myresult=mycursor.fetchall()
        mydb.commit()
        arr.append(myresult)
        #about w price
        sql ="SELECT max(W_price),max(W_price),min(W_price_avg),W_price_remark FROM Product_record,product_img WHERE product_img.Pname=Product_record.Pname and product_img.p_id = %s  GROUP BY W_price_remark"
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(p_id),))
        myresult=mycursor.fetchall()
        mydb.commit()
        arr.append(myresult)
        w_info=myresult
        #about p price
        sql ="SELECT max(P_price),min(P_price),min(P_price_avg),P_price_remark FROM Product_record,product_img WHERE product_img.Pname=Product_record.Pname and product_img.p_id = %s  GROUP BY P_price_remark"
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(p_id),))
        myresult=mycursor.fetchall()
        mydb.commit()
        arr.append(myresult)
        p_info=myresult
        userid=flask.escape(session['userID'])
        #user comment
        sql ="SELECT com_id,comment_tit,comment_msg,u_id,date,Pname FROM comment,product_img WHERE product_img.p_id=comment.p_id and comment.p_id = %s and comment.u_id= %s"
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(p_id),str(userid),))
        myresult=mycursor.fetchall()
        mydb.commit()
        arr.append(myresult)
        return render_template('product_detail.html',data=arr,pid=p_id,w_info=w_info,p_info=p_info)

@app.route("/add_Myf",methods=["POST","GET"])
def add_Myf():
        p_id=request.form.get('p_id')
        userid=flask.escape(session['userID']) 
        sql = "INSERT INTO My_favorite(Pname,u_id) VALUES (%s,%s)"
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(p_id),str(userid),))
        mydb.commit()
        flash('Successfully Added')
        return redirect(url_for('index'))
 
@app.route("/index_Myf",methods=["POST","GET"])
def index_Myf():
        c_day=str(date.today())
        userid=flask.escape(session['userID'])
        sql = " SELECT img_Link,product_img.Pname,W_price,P_price,product_img.p_id FROM Product_record,product_img,My_favorite where product_img.Pname = Product_record.Pname and My_favorite.Pname=product_img.p_id and My_favorite.u_id= %s  and Product_record.lastDate= %s;"         
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(userid),str(c_day)))
        myresult=mycursor.fetchall()
        mydb.commit() 
        return render_template('my_favorite.html',data=myresult)


@app.route("/del_Myf",methods=["POST","GET"])
def del_Myf():
        p_id=request.args.get('id')
        userid=flask.escape(session['userID'])
        sql = " DELETE FROM My_favorite  WHERE  Pname = %s and u_id= %s"         
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(p_id),str(userid),))
        mydb.commit()
        flash('Successfully Deleted')
        return redirect(url_for('index'))

@app.route("/add_MyComment",methods=["POST","GET"])
def add_MyComment():
        p_id=request.form.get('pc_id')
        userid=flask.escape(session['userID'])
        comment=request.form.get('comment')
        comment_tit=request.form.get('comment_tit')
        c_day=str(date.today())
        sql = "INSERT INTO comment(comment_tit,comment_msg,p_id,u_id,date) VALUES (%s,%s,%s,%s,%s)"
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(comment_tit),str(comment),str(p_id),str(userid),str(c_day),))
        mydb.commit()
        flash('Successfully Comented')
        return redirect(url_for('index'))

@app.route("/index_MyComment",methods=["POST","GET"])
def index_MyComment():
        arr=[]
        p_id=request.args.get('p_id')
        userid=flask.escape(session['userID'])
        sql ="SELECT com_id,comment_tit,comment_msg,u_id,date,Pname FROM comment,product_img WHERE product_img.p_id=comment.p_id and comment.p_id = %s "
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(p_id),))
        myresult=mycursor.fetchall()
        mydb.commit()
        arr.append(myresult)
        arr.append(userid)
        return render_template('product_MessageBoard.html',data=arr)
 
@app.route("/del_MyComment",methods=["POST","GET"])
def del_MyComment():
        m_id=request.args.get('id')
        userid=flask.escape(session['userID'])
        sql = " DELETE FROM comment  WHERE  com_id = %s and u_id= %s"         
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(m_id),str(userid),))
        mydb.commit()
        flash('Successfully Deleted')
        return redirect(url_for('index'))   

@app.route("/upd_MyComment",methods=["POST","GET"])
def upd_MyComment():
        comment_tit=request.form.get('comment_tit')
        comment=request.form.get('comment')
        c_id=request.form.get('c_id')
        #userid=flask.escape(session['userID'])
        sql="UPDATE comment SET comment_tit= %s,comment_msg= %s where com_id= %s" 
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(comment_tit),str(comment),str(c_id),))
        mydb.commit()
        flash('Successfully Updated')
        return redirect(url_for('index'))   
    
@app.route("/product_details_chat",methods=["POST","GET"]) 
def product_details_chat():
        arr=[]
        p_id=request.args.get('p_id')
        #new_password= request.form.get('new_Password')
        sql = "SELECT lastDate,W_price,P_price FROM product_img,Product_record WHERE product_img.Pname=Product_record.Pname and product_img.p_id = %s"
        mycursor = mydb.cursor()      
        mycursor.execute(sql,(p_id,))
        myresult = mycursor.fetchall()
        mydb.commit()
        for i in myresult:            
            c_date=str(i[0].month)+'-'+str(i[0].day)
            arr1=c_date,i[1],i[2]
            arr.append(arr1)
        sql = "SELECT Pname FROM product_img WHERE  product_img.p_id = %s"
        mycursor = mydb.cursor()      
        mycursor.execute(sql,(p_id,))
        myresult = mycursor.fetchall()
        mydb.commit()
        product_name=myresult[0][0]
        return render_template('product_line_chart.html',result=arr,name=product_name)

@app.route('/getPlotCSV',methods=["POST","GET"]) 
def plot_csv(): 
        class Record:
          def __init__(self, Pname, Pbrand,W_price,W_price_avg,W_price_remark,W_price_avg_remark,P_price,P_price_avg,P_price_remark,P_price_avg_remark,lastDate):
            self.Pname = Pname
            self.Pbrand = Pbrand
            self.W_price = W_price
            self.W_price_avg = W_price_avg
            self.W_price_remark = W_price_remark
            self.W_price_avg_remark = W_price_avg_remark
            self.P_price = P_price
            self.P_price_avg = P_price_avg
            self.P_price_remark = P_price_remark
            self.P_price_avg_remark = P_price_avg_remark
            self.lastDate = lastDate
        a={}
        a[0]='https://online-price-watch.consumer.org.hk/opw/getPriceList/keyword/Tempo?start=0&length=20&order=asc&sortby=index'
        a[1]='https://online-price-watch.consumer.org.hk/opw/getPriceList/keyword/Lion+%26+Globe?start=0&length=20&order=asc&sortby=index'
        a[2]='https://online-price-watch.consumer.org.hk/opw/getPriceList/keyword/Coca+Cola?start=0&length=20&order=asc&sortby=index'
        a[3]='https://online-price-watch.consumer.org.hk/opw/getPriceList/keyword/Emmi?start=0&length=20&order=asc&sortby=index'
        a[4]='https://online-price-watch.consumer.org.hk/opw/getPriceList/keyword/Golden+Phoenix?start=0&length=20&order=asc&sortby=index'
        a[5]='https://online-price-watch.consumer.org.hk/opw/getPriceList/keyword/Golden+Elephant?start=0&length=20&order=asc&sortby=index'
        data_arr=[]
        for j in a:
          resp = requests.get(a[j])
          soup = BeautifulSoup(resp.text, 'html.parser') 
          result=resp.text
          result_array=json.loads(result)
          result_data=result_array.get('data')
          data = []
          #data_arr=[]
          counter=0  
          for i in result_data:   
            if  'WELLCOME' in result_data[counter]['data'] and  'PARKNSHOP' in result_data[counter]['data'] :
              if result_data[counter]['data']['WELLCOME']['Price'] != "0" and result_data[counter]['data']['PARKNSHOP']['Price'] != "0":
                r1=Record(result_data[counter]['nameEN'],result_data[counter]['brandEN'],result_data[counter]['data']['WELLCOME']['Price'],result_data[counter]['data']['WELLCOME']['PriceAvg'],result_data[counter]['data']['WELLCOME']['PriceRemarkEN'],
                     result_data[counter]['data']['WELLCOME']['PriceAvgRemarkEN'],result_data[counter]['data']['PARKNSHOP']['Price'],result_data[counter]['data']['PARKNSHOP']['PriceAvg'],result_data[counter]['data']['PARKNSHOP']['PriceRemarkEN'],
                     result_data[counter]['data']['PARKNSHOP']['PriceAvgRemarkEN'],result_data[counter]['lastUpdate'])
                counter=counter+1
                data_arr.append(r1)
              else:
                counter=counter+1      
            else:
              counter=counter+1
            
        ct = date.today()
        mouth = ''
        year= ''
        day = ''
        if ct.month <=9: #Automatically add 0 to the month
            mouth = '0'
        if ct.year <=9:
            year = '0'
        if ct.day <=9:
            day = '0'
        now = date.today() #Carry the instant time, used to name the file
        filname=year+str(now.year) + mouth+str(now.month) +day+ str(now.day)+ ".csv" 
        csv_file= open( filname, 'a+', encoding="gbk", newline='')
        writer = csv.writer(csv_file)
        writer.writerow(["Pname", "Pbrand", "W_price", "W_price_avg", "W_price_remark", "W_price_avg_remark", "P_price", "P_price_avg", "P_price_remark", "P_price_avg_remark", "lastDate"])  # 写入标题
        for i in data_arr:    
               writer.writerow([i.Pname,i.Pbrand,i.W_price,i.W_price_avg,i.W_price_remark,i.W_price_avg_remark,i.P_price,i.P_price_avg,i.P_price_remark,i.P_price_avg_remark,i.lastDate])
        csv_file.close()
        return send_file(filname,mimetype='text/csv',attachment_filename=filname,as_attachment=True)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/May/2021 17:58:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2021 17:59:05] "GET /userSignup HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2021 17:59:28] "POST /userSignup HTTP/1.1" 302 -
127.0.0.1 - - [24/May/2021 17:59:32] "GET /index HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2021 17:59:45] "GET /product_detail?id=1 HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2021 17:59:54] "GET /product_details_chat?p_id=1 HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2021 18:00:04] "GET /index_Rice HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2021 18:00:12] "GET /product_detail?id=20 HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2021 18:00:16] "POST /add_Myf HTTP/1.1" 302 -
127.0.0.1 - - [24/May/2021 18:00:20] "GET /index HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2021 18:00:25] "GET /product_detail?id=1 HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2021 18:00:31] "POST /add_Myf HTTP/1.1" 302 -
127.0.0.1 - - [24/May/2021 18:00:34] "GET /index HTTP/1.1" 200 -
127.0.0.1 - -

In [ ]:
        arr=[]
        c_day=str(date.today())
        p_id='26' 
        sql="SELECT img_Link,product_img.Pname,W_price,P_price,product_img.p_id,type,W_price_remark,W_price_avg_remark,P_price_remark,P_price_avg_remark FROM Product_record,product_img where product_img.Pname = Product_record.Pname and Product_record.lastDate= %s and product_img.p_id=%s  "
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(c_day),str(p_id),))
        myresult=mycursor.fetchall()
        mydb.commit()
        arr.append(myresult)
        #about w price
        sql ="SELECT max(W_price),max(W_price),min(W_price_avg),W_price_remark FROM Product_record,product_img WHERE product_img.Pname=Product_record.Pname and product_img.p_id = %s  GROUP BY W_price_remark"
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(p_id),))
        myresult=mycursor.fetchall()
        mydb.commit()
        arr.append(myresult)
        #about p price
        sql ="SELECT max(P_price),min(P_price),min(P_price_avg),P_price_remark FROM Product_record,product_img WHERE product_img.Pname=Product_record.Pname and product_img.p_id = %s  GROUP BY P_price_remark"
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(p_id),))
        myresult=mycursor.fetchall()
        mydb.commit()
        arr.append(myresult)
        userid='1'
        #user comment
        sql ="SELECT com_id,comment_tit,comment_msg,u_id,date,Pname FROM comment,product_img WHERE product_img.p_id=comment.p_id and comment.p_id = %s and comment.u_id= %s"
        mycursor=mydb.cursor()
        mycursor.execute(sql,(str(p_id),str(userid),))
        myresult=mycursor.fetchall()
        mydb.commit()
        arr.append(myresult)
        print(arr[1][0])